In [1]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random



parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="webqsp", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama-3", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
args = parser.parse_args("")


datas, question_string = prepare_dataset(args.dataset)

In [20]:
data = datas[1584]
question = data[question_string]
topics = data['topic_entity']
facts, paths, propagate_list = [], [], []
print(question)

what degrees did barack obama get?


In [21]:
for topic in topics:
    topic_name = topics[topic]
    relations = get_relations(question, topic, topic_name, args, 5)
    paths.append({"entity:": topic_name, "relations": relations})
    entities_id, entities_name = get_entities(topic, relations)
    propagate_list += get_propagate_list(topic_name, relations, entities_name)

In [10]:
facts = propagate(question, propagate_list, args)
prompt = question_prompt.format("\n".join(facts), question)
response = run_llm(prompt, args.temperature, args.max_length, args.openai_api_key, args.llm)
output = {"question": question, "result": response, "path": paths, "prompt": facts}

In [11]:
print(output)

{'question': 'what degrees did barack obama get?', 'result': 'Here are the degrees Barack Obama received:\n\n1. Bachelor of Arts degree in Political Science with a specialization in International Relations\n2. Juris Doctor degree', 'path': [{'entity:': 'Barack Obama', 'relations': ['education.honorary_degree_recipient.honorary_degrees', 'government.politician.government_positions_held', 'people.person.education', 'people.person.employment_history', 'people.person.profession']}], 'prompt': ['Barack Obama has multiple professions, including Politician, Author, Law professor, Writer, and Lawyer.', 'He received an honorary degree from the University of Michigan in 2010.', '(Not relevant to degrees)', 'Barack Obama earned a Bachelor of Arts degree in Political Science with a specialization in International Relations from Columbia University (1981-1983) and a Juris Doctor degree from Harvard Law School (1988-1991).', '(Not relevant to degrees)']}


In [25]:
sparql_head_relations_2hop = """
PREFIX ns: <http://rdf.freebase.com/ns/>
SELECT DISTINCT ?r
WHERE {
ns:%s ns:%s ?e . 
FILTER(?e1 != ns:%s)
?e ?r ?e1 .
}
"""
execute_sparql(sparql_head_relations_2hop % (topic, relations[3], topic))

[{'r': {'type': 'uri',
   'value': 'http://rdf.freebase.com/ns/business.employment_tenure.company'}},
 {'r': {'type': 'uri',
   'value': 'http://rdf.freebase.com/ns/type.object.type'}},
 {'r': {'type': 'uri',
   'value': 'http://rdf.freebase.com/ns/business.employment_tenure.from'}},
 {'r': {'type': 'uri',
   'value': 'http://rdf.freebase.com/ns/business.employment_tenure.title'}},
 {'r': {'type': 'uri',
   'value': 'http://rdf.freebase.com/ns/business.employment_tenure.to'}}]